# CFD Problem's Physics & Theory

**In developing CFD simulation for a fluid mechanics problem understanding the general physics and fundamental theory of the problem is extremely important. This knowledge would provide user the ability to make logical decision about the geometry of CFD domain, choice of proper boundary conditions and numerical models for the CFD simulation setup. Furthermore, user will have a general big picture of the expected results from simulation. These knowledges and abilities form the foundation for a successful and accurate implementation and possibly validation of the CFD simulation.**

**In other words understanding physics and theory of problem before developing a CFD simulation can be thought of turning on a flash light prior to taking any step in a completely dark room. Hence, let's review the fundamental physics and theory behind the problem of "3D Particle Dispersion in Turbulent Flows" as the first step for development and validation of CFD simulation for this problem.**

Particle motion in turbulent flows has extensive applications in field of fluid mechanics. Sediment transport and deposition in channel flows, sewer tanks, boundary and mixing layers and stirred reactors or fluidized bed gasifier are just a few examples of turbulent flows where particles motion dominate the physical behavior of the system. There is significant interest in understanding and modeling particle dispersion mechanisms in turbulent flow fields. Different computational models for simulation of particle dispersion in turbulent flows have been developed. The majority of these models are based on either Eulerian or Lagrangian approaches to simulate the behavior of the dispersed phase. 

Within the Lagrangian approach to simulate particle motion, one of the most widely used models to simulate particle dispersion in a turbulent flow is the Discrete Random Walk (DRW) model. The DRW is a stochastic model first proposed by Hutchinson et al. $[1]$. Dukowicz developed statistical averaging for the DRW to model liquid droplet dispersion in a turbulent flow $[2]$. Later on, the DRW was developed into its current form by Gosman and Ioannides, who modeled the particle-eddy interaction based on an approach different to Dukowicz: in this new approach, individual particles are tracked based on their equations of motion under aero- or hydrodynamic and other external forces (Lagrangian approach) $[3]$. A fundamental model assumption is that particles interact with the fluid, but not with each other (one or two-way coupling), as it is an accepted assumption in low particulate phase volume (and mass) fraction flows, where inter-particle distances are much larger than the diameter, and collisions are statistically insignificant as visualized in Fig. 1:

<img src="./Images/force_blance_viz.png" width="600">
</br>
Fig. 1 - Visualization of force balance on a moving particle within a carrier flow.

The implementation of the DRW model relies on aero- or hydrodynamic forces calculated along the particle trajectories (at the center of mass of each particle in the commonly-used point particle approximation), requiring a statistical description of the carrier flow turbulent field as modeled, for example, by solving the RANS equations with a turbulence closure. The equation of motion of a particle equates the particle inertia with the aero- or hydrodynamic, and other external, forces acting on the particle as follows:

$$\frac{d \vec{V_p}}{dt} = \frac{(\bar{\vec{V}} + \vec{V'(t)})-\vec{V_p}}{\tau_p} + \frac{\vec{g} (\rho_p - \rho)}{\rho_p} + \vec{f}_{other},$$

where $\bar{\vec{V}} + \vec{V}'(t)$ are the mean velocity plus the velocity fluctuations according to Reynolds decomposition, $\rho$ is the density and $\vec{g}$ is the gravitational acceleration, "p" refers to particle variables and the other variables are of the carrier flow. This equation is nothing but Newton's second law of motion. It equates the inertia of the particle to the summation of external forces acting on that particle per it's unit mass; The first term on the right hand side is the drag force per unit particle mass, the second is the gravitational force and the third includes other forces on the particle (i.e. added mass, Basset history, Saffman's lift, phoretic forces, etc.). The DRW model predicts the turbulent dispersion of particles using the instantaneous fluid velocity $\bar{\vec{V}} + \vec{V'}(t)$, along the particle path during the integration of the forces on the particle.  The fluctuating velocity components are assumed to obey a Gaussian probability distribution, with Root Mean Squares given by:

$$  v'_x = \gamma _1 \sqrt{\overline{{u'}^{2}}}, \ \ \ \ \  v'_y = \gamma _2 \sqrt{\overline{{v'}^{2}}}, \ \ \ \ \  v'_z = \gamma _3 \sqrt{\overline{{w'}^{2}}}, $$

where $\gamma _i$ are zero-mean, unit-variance, normally-distributed, random numbers, constant over an interval of time given by the interaction time scale.

Calculation of the eddy lifetime, that is, the frequency with which the velocity fluctuation components are updated as particles move in the turbulent flow field represents the main challenge to accuracy in the DRW model. This time scale is defined as the lowest of two possible particle-eddy interaction times: 

a) The particle interacts with a given eddy till that eddy diffuses away (eddy lifetime is random with mean $(\tau_e)$). At that point, the particle will start interacting with a new eddy and the flow velocity fluctuations on the particle change. This eddy-particle interaction time, which corresponds to the eddy Lagrangian correlation time, is computed in the DRW model as:

<img src="./Images/eddy_lifetime_viz.png" width="400">
</br>
Fig. 2 - Visualization of particle eddy life time scale; particles move to a new eddy after the original eddy diffuses out.

$$\tau_e \approx C_T \frac{k}{\epsilon},$$

where $C_T$ is the constant time scale coefficient at the heart of the problem. $k$ is the turbulent kinetic energy and $\epsilon$ is the turbulent dissipation rate. $C_T$ is a physical parameter, not a fitting constant, relating the life time of the eddies that interact with the particles based on the turbulent kinetic energy and the dissipation rate in the flow. Hence, its value depends on the Reynolds number and type of turbulent flow, and can not be set with mismatched data, due to the scale or the type of flow. Inaccurate values of this time scale constant, $C_T$, will lead to gross under- or overestimation of particles dispersion process, as reported widely by practitioners.

b) The alternative particle-eddy interaction time represents interactions in which the particle crosses, due to inertia or gravitational pull, through an eddy in a time $t_{cross} (< \tau_e)$. The particle will then encounter another eddy and experience different velocity fluctuations. 

<img src="./Images/cross_time_viz.png" width="400">
</br>
Fig. 3 - Visualization of particle cross time scale; particles moves to a new eddy due to their high inertia dominating their interaction with the original eddy.

This particle-eddy interaction time scale can be computed as follows, where $\tau_p$ is the particle relaxation time, $L_e$ is the eddy length scale, and $|\vec{V} -\vec{ V_p}|$ is the magnitude of the relative velocity, given by the gravitational settling velocity in the case of the crossing trajectories effect. 

$$ t_{cross} = -\tau_p~ln[1 - (\frac{L_e}{\tau_p ~|\vec{V} - \vec{V_p}|})], $$

The root of the uncertainty in the implementation of the DRW model to simulate particle dispersion in turbulent flows is the value of the time scale constant $C_T$ in the definition of the particle eddy time scale $\tau_p$. Specially for case studies with small particles in which the eddy life time is the dominant time scale of the flow. This uncertainty has been reported repeatedly in previous works. Ideally, experimental data on the flow of interest would be available for calibration of the DRW model, but the reality is that this is seldom the case in majority of engineering applications, where many cases need to be studied before a prototype is created, or where only small scale tests can be performed to support the design process. A simple calibration procedure is proposed based on theoretical results that apply to high Reynolds number flows. Specifically, the basic idea is that the model needs to reproduce, at least, analytical results for dispersion of particles in homogeneous isotropic turbulence. Only then, can any confidence in the results on a generic high Reynolds number flow be gained. 

To calibrate the correct particle-eddy interaction time-scale constant ($C_T$), the turbulent flow of interest is simplified to an equivalent homogeneous and isotropic flow with the same Reynolds number and similar boundary and initial conditions, in this case study an open channel flow. The dispersion of fluid particles is computed and analyzed for multiple values of $C_T$, and compared to the theoretical from $[4]$:

$$ \sqrt{[X^2]}=\sqrt{2*I*t}*\sqrt{[u^2]} $$.

This equation correlates the Root Mean Squares (RMS) of particle dispersion $\sqrt{[X^2]}$ in the direction of interest to the flow Lagrangian integral correlation time scale $I$, particle residence time $t$ and RMS of flow velocity in the direction of interest $\sqrt{[u^2]}$.

In this case study the steps to develop a CFD simulation for modeling particle dispersion in a turbulent flow using DRW model is presented. The uncertainty of the model is highlighted and calibration methodology and tools for this model is presented in great details. It is believed that using the introduced tools and methodologies will allow users to implement DRW correctly for their specific engineering applications of interest. 


## Source(s)

> [1] Hutchinson, P., Hewitt, G. F., and Dukler, A. E., 1971. **Despersion of liquid or solid dispersions from turbulent gas stream: A stochastic model**. Chemical Engineering Science, 26, pp. 419-439.

> [2] Dukowicz, J. K. , 1980. **A particle-fluid numerical model for liquid sprays.**. Journal of Computational Physics, 35, pp. 229-253.

> [3] Gosman, A. D., and Ioannides, E., 1983. **Aspects of computer simulation of liqud-fueled combustors**. Journal of Energy, (7)-6, pp. 482-490.

> [4] Taylor, G., 1915. **Diffusion by continuous movements**. Eddy motion in the atmosphere, Phil Trans., p. 1.